<div align="right">Peter Norvig 2014</div>

# Sol Golomb’s Rectangle Puzzle

This problem by Solomon Golomb was presented by Gary Antonik in his 14/4/14 New York Times [Numberplay column](http://wordplay.blogs.nytimes.com/2014/04/14/rectangle):

>Say you’re given the following challenge: create a set of five rectangles that have sides of length 1, 2, 3, 4, 5, 6, 7, 8, 9 and 10 units. You can combine sides in a variety of ways: for example, you could create a set of rectangles with dimensions 1 x 3, 2 x 4, 5 x 7, 6 x 8 and 9 x 10.
>
>1. How many different sets of five rectangles are possible?
>
>2. What are the maximum and minimum values for the total areas of the five rectangles?
>
>3. What other values for the total areas of the five rectangles are possible?
>
>4. Which sets of rectangles may be assembled to form a square?

To me, these are interesting questions because, first, I have a (slight) personal connection to Solomon Golomb (my former colleague at USC) and to Nelson Blachman (the father of my colleague Nancy Blachman), who presented the problem to  Antonik, and second, I find it interesting that the problems span the range from mathematical to computational. Let's answer them.

## 1. How many different sets of five rectangles are possible?

This is a basic [combinatorics](http://en.wikipedia.org/wiki/Combinatorics) or counting problem. I will present *three* methods to count the sets. If all goes well they will give the same answer. The example set of rectangles given in the problem was

> {1 &times; 3, 2 &times; 4, 5 &times; 7, 6 &times; 8, 9 &times; 10}
    
and in general it would be

> {A &times; B, C &times; D, E &times; F, G &times; H, I &times; J}

The question is: how many distinct ways can we assign the integers 1 through 10 to the variables A through J?
    
**Method 1: Count all permutations and divide by repetitions:** There are 10 variables to be filled, so there are 10! = 3,628,800 permutations.  But if we fill the first two variables with 1 &times; 3, that is the same rectangle as 3 &times; 1. So divide 10! by 2<sup>5</sup> to account for the fact that each of 5 rectangles can appear 2 ways.  Similarly, if we fill A and B with 1 &times; 3, that yields the same set as if we filled C and D with 1 &times; 3.  So divide again by 5! (the number of permutations of 5 things) to account for this.
That gives us:

> <a href="https://www.google.com/search?ie=UTF-8#q=10!+%2F+2%5E5+/+5!">10! / 2<sup>5</sup> / 5!</a> = 945.  

(It is always a relief when this "count and divide" method comes out to a whole number.)

**Method 2: Count without repetitions**: in each rectangle of the example set the smaller component is listed first, and in each set, the rectangles with smaller first components are listed first.  An alternate to "count and divide" is to count directly how many sets there are that respect this ordering.  We'll work from left to right.  How many choices are there for variable A?  Only one: A must always be 1, because we agreed that the smallest number comes first. Then, given A, there are 9 remaining choices for B.  For C, given A and B, there is again only one choice: C must be the smallest of the remaining 8 numbers (it will be 3 if the first rectangle was 1 &times; 2; otherwise it will be 2, but either way there is only one choice).  That leaves 7 choices for D, 5 for F, 3 for H and 1 for J. So:

> 9 &times; 7 &times; 5 &times; 3 &times; 1 = 945.

(It is always a relief when two methods give the same answer.)
          
**Method 3: Write a program to enumerate the sets:** We'll represent the 1 &times; 3 rectangle as the tuple `(1, 3)` and the example set of rectangles as the set

    {(1, 3), (2, 4), (5, 7), (6, 8), (9, 10)}

We'll write a program to generate all possible sets of rectangles, following method 2, and then just count how many there are. To implement method 2, the minimum side will always be the first element, A, in an (A, B) pair. We iterate through all possible values for B, and then join that pair with all possible rectangles made from the remaining sides. We also have to handle the case when there are no sides; then there is one possible set of rectangles: the empty set.

In [1]:
def rectangle_sets(sides):
    "Given a set of sides, list all distinct sets of rectangles that can be made."
    if not sides:
        return [ set() ]
    else:
        A = min(sides)
        return [ {(A, B)} | other_rects
                for B in sides if B is not A
                for other_rects in rectangle_sets(sides - {A, B}) ]

In [2]:
rectangle_sets({1, 2, 3, 4})

[{(1, 2), (3, 4)}, {(1, 3), (2, 4)}, {(1, 4), (2, 3)}]

We see above that there are three possible rectangle sets for 4 sides (the 1 can pair with any other side, and then a second rectangle can only be made one way from the remaining 2 sides). Below we count how many can be made with 10 sides:

In [3]:
sides = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

all_sets = rectangle_sets(sides)

len(all_sets)

945

(It is a relief that once again we get the same answer, 945.)  

I don't want to print all 945 sets, but let's peek at every 100th one:

In [4]:
all_sets[::100]

[{(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)},
 {(1, 2), (3, 10), (4, 8), (5, 6), (7, 9)},
 {(1, 3), (2, 10), (4, 6), (5, 7), (8, 9)},
 {(1, 4), (2, 10), (3, 5), (6, 8), (7, 9)},
 {(1, 5), (2, 9), (3, 8), (4, 6), (7, 10)},
 {(1, 6), (2, 9), (3, 5), (4, 7), (8, 10)},
 {(1, 7), (2, 9), (3, 4), (5, 8), (6, 10)},
 {(1, 8), (2, 7), (3, 9), (4, 5), (6, 10)},
 {(1, 9), (2, 7), (3, 5), (4, 6), (8, 10)},
 {(1, 10), (2, 7), (3, 4), (5, 8), (6, 9)}]

## 2. What are the maximum and minimum values for the total areas of the five rectangles?

I think I know this one, but I'm not completely sure.  I know that a rectangle with a fixed perimeter has maximum area when it is a square. My guess is that the maximum total area occurs when each rectangle is *almost* square: a 9 &times; 10 rectangle; 7 &times; 8; and so on.  So that would give us a maximum area of:

In [5]:
(1 * 2) + (3 * 4) + (5 * 6) + (7 * 8) + (9  * 10)

190

And the minimum area should be when the rectangles deviate the most from squares: a 1 &times; 10, and so on:

In [6]:
(1 * 10) + (2 * 9) + (3 * 8) + (4 * 7) + (5 * 6)

110

Since I am not sure, I will double check by finding the rectangle sets with the min and max total areas:

In [7]:
def total_area(rectangles): return sum(w * h for (w, h) in rectangles)

In [8]:
max(all_sets, key=total_area)

{(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)}

In [9]:
min(all_sets, key=total_area)

{(1, 10), (2, 9), (3, 8), (4, 7), (5, 6)}

This verifies my intuitions. But I still don't completely understand the situation.  Suppose there are *N* sides that are not  consecutive integers.  Will the maximum total area always be formed by combining the two biggest sides, and so on?

## 3. What other values for the total areas of the five rectangles are possible?

I have no idea how to figure this out mathematically from first principles, but it is easy to compute with the code we already have:

In [10]:
areas = {total_area(s) for s in all_sets}
print(sorted(areas))

[110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 190]


Is there a more succint way to describe these values? Let's see ...

In [11]:
set(range(110, 191)) - areas

{176, 185, 188, 189}

The answer is: "All the integers between 110 and 190 inclusive, except 176, 185, 188, and 189."

## 4. Which sets of rectangles may be assembled to form a square?

The only way I can think about this is to write a program; I don't see any way to work it out by hand.  I do know that the total area will have to be a perfect square, and since the total area is between 110 and 191, that means either 121, 144, or 169:

In [12]:
perfect_squares = {i ** 2 for i in range(100)}

sorted((total_area(s), s) 
       for s in all_sets if total_area(s) in perfect_squares)

[(121, {(1, 6), (2, 9), (3, 10), (4, 8), (5, 7)}),
 (121, {(1, 6), (2, 10), (3, 8), (4, 9), (5, 7)}),
 (121, {(1, 6), (2, 10), (3, 9), (4, 7), (5, 8)}),
 (121, {(1, 7), (2, 10), (3, 6), (4, 9), (5, 8)}),
 (121, {(1, 8), (2, 6), (3, 10), (4, 9), (5, 7)}),
 (121, {(1, 8), (2, 7), (3, 10), (4, 6), (5, 9)}),
 (121, {(1, 8), (2, 9), (3, 7), (4, 6), (5, 10)}),
 (121, {(1, 8), (2, 10), (3, 5), (4, 9), (6, 7)}),
 (121, {(1, 9), (2, 7), (3, 6), (4, 10), (5, 8)}),
 (121, {(1, 9), (2, 7), (3, 8), (4, 6), (5, 10)}),
 (121, {(1, 9), (2, 7), (3, 10), (4, 5), (6, 8)}),
 (121, {(1, 9), (2, 8), (3, 6), (4, 7), (5, 10)}),
 (121, {(1, 10), (2, 5), (3, 9), (4, 8), (6, 7)}),
 (121, {(1, 10), (2, 8), (3, 7), (4, 5), (6, 9)}),
 (144, {(1, 3), (2, 9), (4, 10), (5, 7), (6, 8)}),
 (144, {(1, 3), (2, 10), (4, 7), (5, 9), (6, 8)}),
 (144, {(1, 3), (2, 10), (4, 8), (5, 7), (6, 9)}),
 (144, {(1, 5), (2, 6), (3, 8), (4, 10), (7, 9)}),
 (144, {(1, 7), (2, 4), (3, 8), (5, 9), (6, 10)}),
 (144, {(1, 7), (2, 9), (3, 5),

In [13]:
len(_)

35

So 35 out of 945 rectangle sets *might* be assembled into a square; we don't know yet if they *can* be. 

# Packing Rectangles into a Square

I would like to see *how* the rectangles are packed into the square, not just *which* sets can be packed, so I'll need a data structure to hold information on which rectangle goes where, and a way to visually display the results.
I'll represent a *Grid* as a two-dimensional array: a list of rows, each of which is a list of cells, with the idea that each cell will be covered by a rectangle by assigning `grid[y][x] = rect` (and the special rectangle `empty` will be used to mark a cell as uncovered). I only need to deal with square grids:

In [14]:
empty = (0, 0)

def Square(n):
    "Create an [n x n] grid of empty cells."
    return [[empty for col in range(n)] 
            for row in range(n)]

In [15]:
Square(5)

[[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

Let's make a function to place a single rectangle onto a grid:  `place_rectangle_at((w, h), grid, (x0, y0))` places a rectangle of width *w* and height *h* onto a grid at position (x<sub>0</sub>, y<sub>0</sub>).  If the rectangle overlaps a non-empty cell, or goes off the grid, we return `None` to indicate that this is not a legal placement. If the rectangle does fit, we return a new grid with the new rectangle placed on it (we do not modify the original grid):

In [16]:
def place_rectangle_at(rect, grid, pos):
    """Place the rectangle of size (w, h) onto grid at position (x0, y0).
    Return a new grid, or None if the rectangle cannot be placed."""
    (w, h)   = rect
    (x0, y0) = pos
    newgrid  = [row.copy() for row in grid] 
    for x in range(x0, x0+w):
        for y in range(y0, y0+h):
            if y >= len(grid) or x >= len(grid[y]) or newgrid[y][x] is not empty:
                return None   
            newgrid[y][x] = rect
    return newgrid

In [17]:
# Place a 3 x 4 rectangle on a 5 x 5 square grid, 2 cells over and 1 down:
place_rectangle_at((3, 4), Square(5), (2, 1))

[[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (3, 4), (3, 4), (3, 4)],
 [(0, 0), (0, 0), (3, 4), (3, 4), (3, 4)],
 [(0, 0), (0, 0), (3, 4), (3, 4), (3, 4)],
 [(0, 0), (0, 0), (3, 4), (3, 4), (3, 4)]]

In [18]:
# Place two rectangles on a grid
grid0 = Square(5)
grid1 = place_rectangle_at((3, 4), grid0, (2, 1))
grid2 = place_rectangle_at((2, 5), grid1, (0, 0))
grid2

[[(2, 5), (2, 5), (0, 0), (0, 0), (0, 0)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)]]

In [19]:
# Place a third rectangle that just barely fits
place_rectangle_at((3, 1), grid2, (2, 0))

[[(2, 5), (2, 5), (3, 1), (3, 1), (3, 1)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)]]

In [20]:
# Alternatively, try to place a third rectangle that does not fit
print(place_rectangle_at((4, 1), grid2, (2, 0)))

None


Now we need a strategy for packing a set of rectangles onto a grid.  I know that many variants of [bin packing problems](http://en.wikipedia.org/wiki/Bin_packing_problem) are NP-hard, but we only have 5 rectangles, so it should be easy: just exhaustively try each rectangle in each possible position, and in both possible orientations (horizontal and vertical). But placing rectangles is commutative, so we can do this two ways:

> Way 1: Considering the *rectangles* in a fixed order, try every possible *position* for each *rectangle*.

or

> Way 2: Considering the *positions* in a fixed order, try every possible *rectangle* for each *position*.

In Way 1, we could pre-sort the rectangles (say, biggest first).  Then we try to put the biggest rectangle in all possible positions on the grid, and for each position that fits, try putting the second biggest rectangle in all remaining positions, and so on.  As a rough estimate, assume there are on average about 10 ways to place a rectangle.  Then this way will look at about 10<sup>5</sup> = 100,000 combinations.

In Way 2, we consider the positions in some fixed order; say top-to-bottom, left-to right.  Take the first empty position (say, the upper left corner).  Try putting each of the rectangles there, and for each one that fits, try all possible rectangles in the next empty position, and so on. There are only 5! permutations of rectangles, and each rectangle can go either horizontaly or vertically, so we would have to consider 5! &times; 2<sup>5</sup> = 3840 combinations.  Since 3840 &lt; 100,000, I'll go with Way 2.  Here is a more precise description:

> Way 2: To `pack` a set of rectangles onto a grid, find the first empty cell on the grid.  Try in turn all possible placements of any rectangle (in either orientation) at that position. For each one that fits, try to `pack` the remaining rectangles, and return the resulting grid if one of these packings succeeds.  

In [21]:
def pack(rectangles, grid):
    """Find a way to pack all rectangles onto grid and return the packed grid,
    or return None if not possible."""
    if not rectangles:
        return grid 
    pos = first_empty_cell(grid)
    if grid and pos:
        for (rectangles2, grid2) in rectangle_placements(rectangles, grid, pos):
            solution = pack(rectangles2, grid2)
            if solution:
                return solution

def rectangle_placements(rectangles, grid, pos):
    "Yield all (rect, grid) pairs that result from placing a rectangle at pos on grid."
    for (w, h) in rectangles:
        for rect in [(w, h), (h, w)]:
            grid2 = place_rectangle_at(rect, grid, pos)
            if grid2: 
                yield rectangles - {(w, h)}, grid2       
                
def first_empty_cell(grid):
    "The uppermost, leftmost empty cell."
    for (y, row) in enumerate(grid):
        for (x, cell) in enumerate(row):
            if cell is empty:
                return (x, y)

Let's try a simple example that I know will fit:

In [22]:
pack({(1, 3), (2, 5), (3, 4)}, Square(5))

[[(2, 5), (2, 5), (3, 1), (3, 1), (3, 1)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)],
 [(2, 5), (2, 5), (3, 4), (3, 4), (3, 4)]]

It works! But it is not pretty.

# Colored Rectangles

It would be nicer to have a graphical display of colored rectangles. I will define the function `show` which displays a grid as colored rectangles, by calling upon `html_table`, which formats any grid into HTML text. (*Note:* I used the deprecated `bgcolor` attribute of HTML, rather than CSS styles, because Github is conservative in the javascript and even CSS that it allows, and wouldn't display colored styles.)

In [23]:
from IPython.display import HTML, display, clear_output

def show(grid):
    "Display a colored HTML representation of this grid, unless it is None."
    if grid:
        display(html_table(grid, colored_cell))
    
def html_table(grid, cell_function='<td>{}'.format):
    """Return an HTML <table>, where each cell's contents comes from calling 
    cell_function(grid[y][x])"""
    return HTML('<table>{}</table>'
                .format(cat('\n<tr>' + cat(map(cell_function, row)) 
                        for row in grid)))

def colored_cell(rect): 
    x, y = sorted(rect)
    return '<td bgcolor="{}">{}{}'.format(colors[x], x%10, y%10)

colors = 'lightgrey yellow plum lime cyan coral red olive slateblue wheat'.split()

cat = ''.join

In [24]:
# For example:
show(pack({(1, 3), (2, 5), (3, 4)}, Square(5)))

That's much nicer to look at! Now we can show all the rectangles that are packable into a square:

In [25]:
def pack_square(rectangles):
    "Pack rectangles into a square of appropriate size, if possible."
    A = total_area(rectangles)
    if A in perfect_squares:
        return pack(rectangles, Square(int(A ** 0.5)))

for s in all_sets:
    show(pack_square(s))

So there are 4 sets of rectangles that work.

# Animated Colored Rectangles

It is gratifying to see the final results, and have the computation be so fast, but I'd like to get a better understanding of the algorithm.  I can visualize the process by *animating* the search that `pack` does: on every recursive call to `pack`, I can show the grid, and then pause briefly. I do that by redefining `pack`; the only change is the line starting `if pause:`.

In [26]:
import time

def pack(rectangles, grid, pause=None):
    """Find a way to pack all rectangles onto grid and return the packed grid,
    or return None if not possible."""
    if pause: (clear_output(), show(grid), time.sleep(pause))
    if not rectangles:
        return grid 
    pos = first_empty_cell(grid)
    if grid and pos:
        for (rectangles2, grid2) in rectangle_placements(rectangles, grid, pos):
            solution = pack(rectangles2, grid2, pause)
            if solution:
                return solution

If you are running this in a live IPython notebook (not in nbviewer), you can see for yourself by re-running this cell:

In [27]:
pack({(5, 10), (3, 6), (4, 7), (2, 8), (1, 9)}, Square(11), 1);